<a href="https://colab.research.google.com/github/godmin18/project/blob/master/Xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://www.dacon.io/competitions/open/235670/codeshare/1911?page=2&dtype=recent
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWiDgX47LaLL1pkFsYN6gs619LlBNaYFX2IYjO6UDo7qybuBoqTD5_k
Mounted at /content/drive


In [2]:
#GPU 사용 체크
import torch
import torchvision
import torch.nn
torch.cuda.is_available()

False

In [3]:
import pandas as pd
import numpy as np
import re
import os
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
DIR = '/content/drive/MyDrive/project/소설 작가 분류 AI 경진대회/dataset'
TRAIN_SOURCE = os.path.join(DIR, "train.csv")
TEST_SOURCE = os.path.join(DIR, "test_x.csv")
SAMPLE_SUBMISSION = os.path.join(DIR, "sample_submission.csv")

train = pd.read_csv(TRAIN_SOURCE)
test = pd.read_csv(TEST_SOURCE)
print("Number of rows in train dataset : ",train.shape[0])
print("Number of rows in test dataset : ",test.shape[0])

Number of rows in train dataset :  54879
Number of rows in test dataset :  19617


In [5]:
#문장으로부터 pure word를 추출 및 단어의 개수
def clean(train,test):
    train['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in train['text']]
    test['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in test['text']]
    return train,test
train, tst = clean(train, test)

train["num_words"] = train["text"].apply(lambda x: len(str(x).split()))
test["num_words"] = test["text"].apply(lambda x: len(str(x).split()))

In [6]:
train

,index,text,author,words,num_words
0,0,"He was almost choking. There was so much, so m...",3,"[he, was, almost, choking, there, was, so, muc...",46
1,1,"“Your sister asked for it, I suppose?”",2,"[your, sister, asked, for, it, i, suppose]",7
2,2,"She was engaged one day as she walked, in per...",1,"[she, was, engaged, one, day, as, she, walked,...",57
3,3,"The captain was in the porch, keeping himself ...",4,"[the, captain, was, in, the, porch, keeping, h...",58
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3,"[have, mercy, gentlemen, odin, flung, up, his,...",39
...,...,...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2,"[is, that, you, mr, smith, odin, whispered, i,...",15
54875,54875,"I told my plan to the captain, and between us ...",4,"[i, told, my, plan, to, the, captain, and, bet...",18
54876,54876,"""Your sincere well-wisher, friend, and sister...",1,"[your, sincere, well, wisher, friend, and, sis...",8
54877,54877,“Then you wanted me to lend you money?”,3,"[then, you, wanted, me, to, lend, you, money]",8


In [7]:

#각 저자마다 좋아하는 구두점의 개수와 비율을 가진 피쳐 생성(string 내장함수와 직접 선언)

train["num_punct"] = train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test["num_punct"] = test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

punctuations = [{"id":1,"p":"[;:]"},{"id":2,"p":"[,.]"},{"id":3,"p":"[?]"},{"id":4,"p":"[\']"},{"id":5,"p":"[\"]"},{"id":6,"p":"[;:,.?\'\"]"}]
for p in punctuations:
    punctuation = p["p"]
    temp_trn =  [ sentence.split() for sentence in train['text'] ]
    train['punc_'+str(p["id"])] = [len([word for word in sentence if bool(re.search(punctuation, word))])*100.0/len(sentence) for sentence in temp_trn]    
    temp_tst =  [ sentence.split() for sentence in test['text'] ]
    test['punc_'+str(p["id"])] = [len([word for word in sentence if bool(re.search(punctuation, word))])*100.0/len(sentence) for sentence in temp_tst]

In [ ]:
#각 문장의 stopword 개수와 비율을 추출
#0으로 나뉘어지는 경우는 전체 길이에 각각 1을 더해서 해결
train["num_stopwords"] = train["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in set(stopwords.words('english'))]))
test["num_stopwords"] = test["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in set(stopwords.words('english'))]))

stop_train = [x for x in train['words']]
train['stop_word'] = [len([word for word in sentence if word in stopwords.words('english')])*100.0/(len(sentence)+1) for sentence in stop_trn]

stop_test = [x for x in test['words']]
test['stop_word'] = [len([word for word in sentence if word in stopwords.words('english')])*100.0/(len(sentence)+1) for sentence in stop_tst] 

In [ ]:
## 그 외 특정단어, 대문자 단어 등의 개수를 추출
train["num_unique_words"] = train["text"].apply(lambda x: len(set(str(x).split())))
test["num_unique_words"] = test["text"].apply(lambda x: len(set(str(x).split())))

train["num_chars"] = train["text"].apply(lambda x: len(str(x)))
test["num_chars"] = test["text"].apply(lambda x: len(str(x)))

train["num_words_upper"] = train["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test["num_words_upper"] = test["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

train["num_words_title"] = train["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test["num_words_title"] = test["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

train["mean_word_len"] = train["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test["mean_word_len"] = test["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
train

,index,text,author,words,num_words,num_punct,punc_1,punc_2,punc_3,punc_4,punc_5,punc_6
0,0,"He was almost choking. There was so much, so m...",3,"[he, was, almost, choking, there, was, so, muc...",46,8,2.173913,15.217391,0.000000,0.000000,0.0,17.391304
1,1,"“Your sister asked for it, I suppose?”",2,"[your, sister, asked, for, it, i, suppose]",7,2,0.000000,14.285714,14.285714,0.000000,0.0,28.571429
2,2,"She was engaged one day as she walked, in per...",1,"[she, was, engaged, one, day, as, she, walked,...",57,9,1.754386,14.035088,0.000000,0.000000,0.0,15.789474
3,3,"The captain was in the porch, keeping himself ...",4,"[the, captain, was, in, the, porch, keeping, h...",58,18,3.448276,25.862069,0.000000,1.724138,0.0,31.034483
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3,"[have, mercy, gentlemen, odin, flung, up, his,...",39,13,2.564103,17.948718,0.000000,0.000000,0.0,20.512821
...,...,...,...,...,...,...,...,...,...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2,"[is, that, you, mr, smith, odin, whispered, i,...",15,5,0.000000,26.666667,6.666667,0.000000,0.0,33.333333
54875,54875,"I told my plan to the captain, and between us ...",4,"[i, told, my, plan, to, the, captain, and, bet...",18,2,0.000000,11.111111,0.000000,0.000000,0.0,11.111111
54876,54876,"""Your sincere well-wisher, friend, and sister...",1,"[your, sincere, well, wisher, friend, and, sis...",8,7,0.000000,50.000000,0.000000,0.000000,25.0,75.000000
54877,54877,“Then you wanted me to lend you money?”,3,"[then, you, wanted, me, to, lend, you, money]",8,1,0.000000,0.000000,12.500000,0.000000,0.0,12.500000
